In [15]:
#Add Dependencies
import pandas as pd
import numpy as np
import psycopg2 as pg
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

### Connection to the Postgres database and Retrive data from the tables

In [16]:
#Read Data from the PostgressSql
conn = pg.connect(user = "postgres",
                                  password = "akshaj10",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "movie_data")
sql1 = "SELECT * FROM oscar_metadata;"
combined_df = pd.read_sql_query(sql1, conn)
conn = None

In [17]:
#Load Data into Pandas Data frame and Display the dataframe.
combined_df.shape

(51147, 17)

In [18]:
# Filter all rows for US as the country
movie_us_df = combined_df[combined_df['country'] == 'United States of America'] 
movie_us_df.shape

(22647, 17)

In [19]:
# Find null values
for column in movie_us_df.columns:
    print(f"Column{column} has {movie_us_df[column].isnull().sum()} null values")

Columnimdb_id has 3 null values
Columnbudget has 0 null values
Columnoriginal_title has 0 null values
Columnpopularity has 0 null values
Columnrevenue has 0 null values
Columnruntime has 5 null values
Columntitle has 0 null values
Columnvote_average has 0 null values
Columnvote_count has 0 null values
Columnproduction has 0 null values
Columncountry has 0 null values
Columnnew_genre has 0 null values
ColumnRelease_year has 2 null values
Columnindex has 15911 null values
Columncategory has 15911 null values
Columnfilm has 15911 null values
Columnwinner has 15911 null values


In [20]:
#Set the index to IMDB_ID
movie_us_df = movie_us_df.set_index("imdb_id")
movie_us_df.head()

,budget,original_title,popularity,revenue,runtime,title,vote_average,vote_count,production,country,new_genre,Release_year,index,category,film,winner
imdb_id,,,,,,,,,,,,,,,,
tt0114709,30000000,Toy Story,30000000.0,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995,7548.0,WRITING (Screenplay Written Directly for the S...,Toy Story,False
tt0114709,30000000,Toy Story,30000000.0,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995,7514.0,MUSIC (Original Song),Toy Story,False
tt0114709,30000000,Toy Story,30000000.0,373554033.0,81.0,Toy Story,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,1995,7508.0,MUSIC (Original Musical or Comedy Score),Toy Story,False
tt0113497,65000000,Jumanji,65000000.0,262797249.0,104.0,Jumanji,6.9,2413.0,TriStar Pictures,United States of America,Adventure,1995,NaN,None,None,None
tt0113228,0,Grumpier Old Men,0.0,0.0,101.0,Grumpier Old Men,6.5,92.0,Warner Bros.,United States of America,Romance,1995,NaN,None,None,None


In [21]:
columns =['budget','original_title','popularity','revenue','runtime','vote_average','vote_count','production','country','new_genre','Release_year', 'winner']

target = ['winner']

In [22]:
movie_us_df = movie_us_df.loc[:, columns].copy()
movie_us_df.shape

(22647, 12)

In [23]:
#Convert winner column into numbers 
movie_us_df["winner"] = movie_us_df["winner"].replace({True: 1, False: 0})
movie_us_df.shape

(22647, 12)

In [24]:
#Fill null values in winner column to zero 
movie_us_df["winner"] = movie_us_df["winner"].fillna(value=0)
movie_us_df.shape

(22647, 12)

In [25]:
# convert the 'Date' column to datetime format 
movie_us_df['Release_year'] = movie_us_df['Release_year'].astype('datetime64[ns]')

In [26]:
# Extract year from date 
movie_us_df['Release_year'] = pd.DatetimeIndex(movie_us_df['Release_year']).year
movie_us_df.shape

(22647, 12)

In [27]:
# Filter all rows for movies from 1990
movie_us_df = movie_us_df[movie_us_df['Release_year'] >= 1990] 
movie_us_df.shape

(11828, 12)

In [28]:
# Drop the Release year after filtering the data.
movie_us_df = movie_us_df.drop('Release_year',1)

In [30]:
movie_us_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11828 entries, tt0114709 to tt0303758
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   budget          11828 non-null  int64  
 1   original_title  11828 non-null  object 
 2   popularity      11828 non-null  float64
 3   revenue         11828 non-null  float64
 4   runtime         11824 non-null  float64
 5   vote_average    11828 non-null  float64
 6   vote_count      11828 non-null  float64
 7   production      11828 non-null  object 
 8   country         11828 non-null  object 
 9   new_genre       11828 non-null  object 
 10  winner          11828 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.1+ MB


In [31]:
# Create a title Dataframe.
title_df=pd.DataFrame(data=movie_us_df['original_title'],index=movie_us_df.index,copy=True)
title_df.head()

,original_title
imdb_id,
tt0114709,Toy Story
tt0114709,Toy Story
tt0114709,Toy Story
tt0113497,Jumanji
tt0113228,Grumpier Old Men


In [32]:
# Remove the movie title from the dataframe
movie_us_df = movie_us_df.drop('original_title',1)
movie_us_df.head()

,budget,popularity,revenue,runtime,vote_average,vote_count,production,country,new_genre,winner
imdb_id,,,,,,,,,,
tt0114709,30000000,30000000.0,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,0.0
tt0114709,30000000,30000000.0,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,0.0
tt0114709,30000000,30000000.0,373554033.0,81.0,7.7,5415.0,Pixar Animation Studios,United States of America,Animation,0.0
tt0113497,65000000,65000000.0,262797249.0,104.0,6.9,2413.0,TriStar Pictures,United States of America,Adventure,0.0
tt0113228,0,0.0,0.0,101.0,6.5,92.0,Warner Bros.,United States of America,Romance,0.0


In [33]:
# Drop null rows
movie_us_df = movie_us_df.dropna(axis = 0,how= 'any')
movie_us_df.shape

(11824, 10)

In [34]:
# Generate our categorical variable list for movie dataset
movie_cat = movie_us_df.dtypes[movie_us_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
movie_us_df[movie_cat].nunique()

production    3323
country          1
new_genre       20
dtype: int64

### One Hot encoding production -Using top 10 values

In [35]:
# Check the production unique value counts to see if binning is required
production_counts= movie_us_df.production.value_counts()
production_counts

Other                        1764
Paramount Pictures            409
Universal Pictures            318
Columbia Pictures             283
Walt Disney Pictures          247
                             ... 
185 Trax                        1
Rubber Tree Productions         1
Applecreek Productions          1
Lucky UKFS                      1
Sound Venture Productions       1
Name: production, Length: 3323, dtype: int64

In [36]:
# Determine which values to replace for bucketing
replace_production = list(production_counts[production_counts < 80].index)

# Replace in DataFrame
for production in replace_production:
    movie_us_df.production = movie_us_df.production.replace(production,"Other")


# Check to make sure binning was successful
movie_us_df.production.value_counts()

Other                                     9542
Paramount Pictures                         409
Universal Pictures                         318
Columbia Pictures                          283
Walt Disney Pictures                       247
Twentieth Century Fox Film Corporation     216
New Line Cinema                            212
Miramax Films                              207
Warner Bros.                               109
TriStar Pictures                           108
DreamWorks SKG                              93
Fox Searchlight Pictures                    80
Name: production, dtype: int64

In [37]:
# let's examine how many columns we will obtain after one hot encoding these variables
encode_df=pd.get_dummies(movie_us_df[movie_cat], drop_first=True)
encode_df.shape

(11824, 30)

In [38]:
# Merge one-hot encoded features and drop the originals
oscar_us_df = movie_us_df.merge(encode_df,left_index=True, right_index=True)
oscar_us_df = oscar_us_df.drop(movie_cat,1)
#oscar_us_df = oscar_us_df.drop('Release_year',1)
oscar_us_df.head(2)

,budget,popularity,revenue,runtime,vote_average,vote_count,winner,production_DreamWorks SKG,production_Fox Searchlight Pictures,production_Miramax Films,...,new_genre_History,new_genre_Horror,new_genre_Music,new_genre_Mystery,new_genre_Romance,new_genre_Science Fiction,new_genre_TV Movie,new_genre_Thriller,new_genre_War,new_genre_Western
imdb_id,,,,,,,,,,,,,,,,,,,,,
tt0035423,0,0.0,0.0,118.0,6.0,430.0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
tt0081145,0,0.0,0.0,94.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Classification (RandomForestClassification)

In [39]:
# Define the features set.
X = oscar_us_df.copy()
X = X.drop('winner', axis=1)
X.shape

(19904, 36)

In [40]:
# Define the target set(output label)
y = oscar_us_df['winner']
y.head(2)

imdb_id
tt0035423    0.0
tt0081145    0.0
Name: winner, dtype: float64

In [41]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [42]:
# Creating a StandardScaler instance.
scaler = StandardScaler()

In [43]:
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

In [44]:
# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [45]:
# Create a random forest classifier.
#rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [46]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [47]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)
predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [48]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,4143,156
Actual 1,444,233


In [49]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)
acc_score

0.8794212218649518

In [50]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4143,156
Actual 1,444,233


Accuracy Score : 0.8794212218649518
Classification Report
              precision    recall  f1-score   support

         0.0       0.90      0.96      0.93      4299
         1.0       0.60      0.34      0.44       677

    accuracy                           0.88      4976
   macro avg       0.75      0.65      0.68      4976
weighted avg       0.86      0.88      0.87      4976



In [51]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([9.33595009e-02, 9.15798275e-02, 1.52665869e-01, 1.77077639e-01,
       1.43205650e-01, 1.84338548e-01, 3.11525578e-03, 2.22103676e-03,
       1.47627058e-02, 2.12728499e-03, 1.70631741e-02, 1.57597967e-02,
       1.82047777e-03, 6.86745727e-03, 4.46546976e-03, 3.50832681e-03,
       1.71917787e-03, 1.25803911e-02, 9.63753593e-03, 7.08878473e-03,
       4.60398302e-03, 3.67483988e-03, 1.81342854e-02, 1.03718725e-03,
       3.60643551e-03, 9.39261715e-07, 1.01484362e-03, 2.55836743e-03,
       2.37214699e-03, 1.74273456e-03, 3.49612584e-03, 1.28336432e-03,
       4.86652663e-03, 5.09271749e-03, 3.76341523e-04, 1.17525189e-03])

In [52]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.18433854764705956, 'vote_count'),
 (0.17707763906416982, 'runtime'),
 (0.1526658689885408, 'revenue'),
 (0.14320564995753832, 'vote_average'),
 (0.09335950087750435, 'budget'),
 (0.0915798275071942, 'popularity'),
 (0.018134285442311326, 'new_genre_Drama'),
 (0.01706317405537106, 'production_Other'),
 (0.015759796686668232, 'production_Paramount Pictures'),
 (0.014762705755839281, 'production_Miramax Films'),
 (0.012580391116892756, 'new_genre_Adventure'),
 (0.009637535934967195, 'new_genre_Animation'),
 (0.007088784726072476, 'new_genre_Comedy'),
 (0.006867457267840792, 'production_Twentieth Century Fox Film Corporation'),
 (0.005092717494977402, 'new_genre_Thriller'),
 (0.004866526625871406, 'new_genre_TV Movie'),
 (0.004603983022653402, 'new_genre_Crime'),
 (0.004465469764734304, 'production_Universal Pictures'),
 (0.003674839877651176, 'new_genre_Documentary'),
 (0.0036064355084154745, 'new_genre_Fantasy'),
 (0.003508326813961853, 'production_Walt Disney Pictures'),
 (0.0034961